In [1]:
import ibis
import ibis_ml as ml
from ibis import _

ibis.options.interactive = True

Let's pick up where we left off by reloading our model input table.

In [2]:
model_input_table = ibis.read_parquet("model_input_table.parquet")
model_input_table

┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ game_id  ┃ is_rated ┃ lichess_time_control_type ┃ is_tournament ┃ white_elo ┃ black_elo ┃ elo_diff ┃ white_elo_gained_since_previous_game ┃ black_elo_gained_since_previous_game ┃ white_title ┃ black_title ┃ base_time ┃ increment ┃ target  ┃ ply   ┃ mate  ┃ cp      ┃ white_clock ┃ black_clock ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ boolean  │ string                    │ boolean       │ int64     │ int64     │ int64    │ int64                                │ int64                                │ string      │ string      │ int64     │ int64     │ float64 │ int64 │ int64 │ float64 │ float64     │ float64     │
├──────────┼──────────┼───────────────────────────┼───────────────┼───────────┼───────────┼──────────┼──────────────────────────────────────┼──────────────────────────────────────┼─────────────┼─────────────┼───────────┼───────────┼─────────┼───────┼───────┼─────────┼─────────────┼─────────────┤
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     1 │  NULL │    0.18 │       300.0 │       300.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     2 │  NULL │    0.21 │       300.0 │       600.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     4 │  NULL │    0.17 │       291.0 │       595.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     5 │  NULL │    0.15 │       290.0 │       595.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     6 │  NULL │    0.22 │       290.0 │       592.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     8 │  NULL │    0.14 │       286.0 │       589.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     9 │  NULL │    0.07 │       276.0 │       589.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │    11 │  NULL │    0.03 │       267.0 │       586.0 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91

# Data splitting

To get started, let's split this single dataset into two: a _training_ set and a _testing_ set. We'll keep most of the rows in the original dataset (subset chosen randomly) in the _training_ set. The training data will be used to _fit_ the model, and the _testing_ set will be used to measure model performance.

Because the order of rows in an Ibis table is undefined, we need a unique key to split the data reproducibly. To ensure that moves corresponding to a particular game aren't split across the _training_ and _testing_ sets, we'll only split by `game_id` (instead of splitting by `game_id` and `ply`).

In [3]:
# Create data frames for the two sets:
train_data, test_data = ml.train_test_split(
    model_input_table,
    unique_key="game_id",
    # Put 3/4 of the data into the training set
    test_size=0.25,
    num_buckets=4,
    # Fix the random numbers by setting the seed
    # This enables the analysis to be reproducible when random numbers are used
    random_seed=222,
)

/workspaces/lichess-live-win-probability-tutorial/.venv/lib/python3.12/site-packages/ibis/expr/types/relations.py:685: FutureWarning: Selecting/filtering arbitrary expressions in `Table.__getitem__` is deprecated and will be removed in version 10.0. Please use `Table.select` or `Table.filter` instead.
  warnings.warn(


# Create features

In [4]:
MATE_SCORE = 100_000

calculate_score_step = ml.Mutate(
    score=ibis.coalesce(
        (_.cp * 100).round(),
        _.mate.sign() * MATE_SCORE - _.mate.try_cast(int) * MATE_SCORE // 100,
        _.target.case().when(1.0, MATE_SCORE).when(0.0, -MATE_SCORE).when(0.5, 0).end(),
    )
)

In [5]:
calculate_score_step.fit_table(train_data, ml.core.Metadata())
calculate_score_step.transform_table(train_data)

┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━┓
┃ game_id  ┃ is_rated ┃ lichess_time_control_type ┃ is_tournament ┃ white_elo ┃ black_elo ┃ elo_diff ┃ white_elo_gained_since_previous_game ┃ black_elo_gained_since_previous_game ┃ white_title ┃ black_title ┃ base_time ┃ increment ┃ target  ┃ ply   ┃ mate  ┃ cp      ┃ white_clock ┃ black_clock ┃ score ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━┩
│ string   │ boolean  │ string                    │ boolean       │ int64     │ int64     │ int64    │ int64                                │ int64                                │ string      │ string      │ int64     │ int64     │ float64 │ int64 │ int64 │ float64 │ float64     │ float64     │ int64 │
├──────────┼──────────┼───────────────────────────┼───────────────┼───────────┼───────────┼──────────┼──────────────────────────────────────┼──────────────────────────────────────┼─────────────┼─────────────┼───────────┼───────────┼─────────┼───────┼───────┼─────────┼─────────────┼─────────────┼───────┤
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     1 │  NULL │    0.18 │       300.0 │       300.0 │    18 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     2 │  NULL │    0.21 │       300.0 │       600.0 │    21 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     4 │  NULL │    0.17 │       291.0 │       595.0 │    17 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     5 │  NULL │    0.15 │       290.0 │       595.0 │    15 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     6 │  NULL │    0.22 │       290.0 │       592.0 │    22 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     8 │  NULL │    0.14 │       286.0 │       589.0 │    14 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │     9 │  NULL │    0.07 │       276.0 │       589.0 │     7 │
│ 6WBbPWDJ │ True     │ Rapid                     │ True          │      1591 │      1342 │      249 │                                   91 │                                 NULL │ NULL        │ NULL        │       600 │         0 │     0.0 │    11 │  NULL │    0.03 │       267.0 │       586.0 │     3 │
│ 6WBbPWDJ │ True     │ Rapid      

In [ ]:
lichess_rec = ml.Recipe(
    # calculate_score_step,  # TODO(deepyaman): Compute score without target, or calculate in feature engineering notebook
    ml.Mutate(
        etime=_.base_time + _.increment * 2 / 3,
        white_etime=_.white_clock + _.increment * 2 / 3,
        black_etime=_.black_clock + _.increment * 2 / 3,
    ),
    ml.Drop(ml.nominal()),
    ml.Cast(ml.everything(), "float64"),
)

# Fit a model with a recipe

In [7]:
import xgboost as xgb
from sklearn.pipeline import Pipeline

pipe = Pipeline([("lichess_rec", lichess_rec), ("clf", xgb.XGBClassifier())])

In [8]:
X_train = train_data.drop("target")
y_train = train_data.target * 2
pipe.fit(X_train, y_train)

Pipeline(steps=[('lichess_rec',
                 Recipe(Mutate(etime=(_.base_time + ((_.increment * 2) / 3)),
                               white_etime=(_.white_clock + ((_.increment * 2) / 3)),
                               black_etime=(_.black_clock + ((_.increment * 2) / 3))),
                        Drop(nominal()), Cast(everything(), 'float64'))),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_b...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

# Use a trained workflow to predict

In [9]:
X_test = test_data.drop("target")
y_test = test_data.target * 2
pipe.score(X_test, y_test)

0.674447454333161